In [ ]:
import cv2
import numpy as np
import random
import os

generated_dir = "generated_cel"

def generate_random_cells_image(
    image_size=(512, 512),
    min_cells=10,
    max_cells=30,
    radius_range=(10, 30)
):
    h, w = image_size
    background = np.full((h, w, 3), (177, 177, 227), dtype=np.uint8)
    annotations = []

    num_cells = random.randint(min_cells, max_cells)

    for _ in range(num_cells):
        radius = random.randint(*radius_range)
        x_center = random.randint(radius, w - radius)
        y_center = random.randint(radius, h - radius)

        color = (
            random.randint(100, 255),  # R
            random.randint(0, 100),    # G
            random.randint(0, 100)     # B
        )

        cv2.circle(background, (x_center, y_center), radius, color, -1)

        annotations.append({
            "x_center": x_center,
            "y_center": y_center,
            "radius": radius
        })

    return background, annotations


In [66]:
def save_image_and_annotations(image, annotations, output_dir, index):
    os.makedirs(output_dir, exist_ok=True)
    img_path = os.path.join(output_dir, f"cell_img_{index}.png")

    cv2.imwrite(img_path, image)


for i in range(20):
    image, annotations = generate_random_cells_image()
    save_image_and_annotations(image, annotations, generated_dir, i)


In [70]:
def extract_cells_and_background(image_path, output_cells_img_path, output_bg_img_path):
    image = cv2.imread(image_path)
    original = image.copy()

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)

    circles = cv2.HoughCircles(
        blurred, 
        cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=50, param2=30, minRadius=10, maxRadius=40
    )

    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cells_only = np.ones_like(image) * 255

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")

        for (x, y, r) in circles:
            cell_mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.circle(cell_mask, (x, y), r, 255, -1)

            cell = cv2.bitwise_and(original, original, mask=cell_mask)

            cells_only[cell_mask == 255] = cell[cell_mask == 255]

            cv2.circle(mask, (x, y), r, 255, -1)

    background = original.copy()
    background[mask == 255] = 255

    cv2.imwrite(output_cells_img_path, cells_only)
    cv2.imwrite(output_bg_img_path, background)

    print(f"Сохранено: клетки -> {output_cells_img_path}, фон -> {output_bg_img_path}")


In [74]:
image_files = [f for f in os.listdir(generated_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
i = 0
for image_file in image_files:
    image_path = os.path.join(generated_dir, image_file)
    extract_cells_and_background(
        image_path=image_path,
        output_cells_img_path=f"cropped_cells/cell_{i}.png",
        output_bg_img_path=f"backgrounds/bg_{i}.png"
    )
    i+=1

Сохранено: клетки -> cropped_cells/cell_0.png, фон -> backgrounds/bg_0.png
Сохранено: клетки -> cropped_cells/cell_1.png, фон -> backgrounds/bg_1.png
Сохранено: клетки -> cropped_cells/cell_2.png, фон -> backgrounds/bg_2.png
Сохранено: клетки -> cropped_cells/cell_3.png, фон -> backgrounds/bg_3.png
Сохранено: клетки -> cropped_cells/cell_4.png, фон -> backgrounds/bg_4.png
Сохранено: клетки -> cropped_cells/cell_5.png, фон -> backgrounds/bg_5.png
Сохранено: клетки -> cropped_cells/cell_6.png, фон -> backgrounds/bg_6.png
Сохранено: клетки -> cropped_cells/cell_7.png, фон -> backgrounds/bg_7.png
Сохранено: клетки -> cropped_cells/cell_8.png, фон -> backgrounds/bg_8.png
Сохранено: клетки -> cropped_cells/cell_9.png, фон -> backgrounds/bg_9.png
Сохранено: клетки -> cropped_cells/cell_10.png, фон -> backgrounds/bg_10.png
Сохранено: клетки -> cropped_cells/cell_11.png, фон -> backgrounds/bg_11.png
Сохранено: клетки -> cropped_cells/cell_12.png, фон -> backgrounds/bg_12.png
Сохранено: клетки -